<a href="https://colab.research.google.com/github/zipo0505/MNIST/blob/main/%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:

from torch import nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Sequential(
        						nn.Conv2d(1,16,kernel_size=3) ,
                                nn.BatchNorm2d(16) ,
                                nn.ReLU(inplace=True))

        self.layer2 = nn.Sequential(
        						nn.Conv2d(16,32,kernel_size=3) ,
                                nn.BatchNorm2d(32) ,
                                nn.ReLU(inplace=True) ,
                                nn.MaxPool2d(kernel_size=2 , stride=2))

        self.layer3 = nn.Sequential(
        						nn.Conv2d(32,64,kernel_size=3) ,
                                nn.BatchNorm2d(64) ,
                                nn.ReLU(inplace=True))

        self.layer4 = nn.Sequential(
        						nn.Conv2d(64,128,kernel_size=3) ,
                                nn.BatchNorm2d(128) ,
                                nn.ReLU(inplace=True) ,
                                nn.MaxPool2d(kernel_size=2 , stride=2))

        self.fc = nn.Sequential(nn.Linear(128*4*4,1024) ,
                                nn.ReLU(inplace=True) ,
                                nn.Linear(1024,128) ,
                                nn.ReLU(inplace=True) ,
                                nn.Linear(128,10) )
    def forward( self , x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # x = x.view(x.size(0) , -1)
        x = x.reshape(x.size(0) , -1)
        fc_out = self.fc(x)
        return fc_out



In [15]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 定義一個簡單的卷積神經網路
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # 展平張量
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 定義超參數
learning_rate = 1e-2      # 學習率
batch_size = 128          # 批的大小
epoches_num = 20          # 遍歷訓練集的次數

# 下載並加載 MNIST 手寫數字訓練集
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 定義模型、損失函數、優化器
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 檢查是否可以使用 GPU
if torch.cuda.is_available():
    print("CUDA is enabled!")
    model = model.cuda()
model.train()

# 開始訓練
for epoch in range(epoches_num):
    print("=" * 40)
    train_loss = 0.0
    train_acc = 0.0

    # 逐批次訓練
    for i, data in enumerate(train_loader, 1):
        img, label = data

        # 使用 GPU 加速（若可行）
        if torch.cuda.is_available():
            img = img.cuda()
            label = label.cuda()

        # 前向傳播
        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, label)

        # 反向傳播
        loss.backward()
        optimizer.step()

        # 計算損失和準確率
        train_loss += loss.item() * label.size(0)
        _, pred = out.max(1)
        num_correct = pred.eq(label).sum().item()
        train_acc += num_correct

    # 輸出每回合的平均損失和準確率
    print('Finish {} epoch: Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, train_loss / len(train_dataset), train_acc / len(train_dataset)))

# 保存模型
torch.save(model, 'cnn.pt')

# 測試模型
# 下載並加載 MNIST 測試集
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 加載訓練完成的模型
model = torch.load('cnn.pt')
criterion = nn.CrossEntropyLoss()
model.eval()
eval_acc = 0
eval_loss = 0

# 測試
for data in test_loader:
    img, label = data
    if torch.cuda.is_available():
        img = img.cuda()
        label = label.cuda()

    # 前向傳播
    out = model(img)
    loss = criterion(out, label)
    eval_loss += loss.item() * label.size(0)

    # 計算準確率
    _, pred = out.max(1)
    num_correct = pred.eq(label).sum().item()
    eval_acc += num_correct

# 輸出測試損失和準確率
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / len(test_dataset), eval_acc / len(test_dataset)))


Finish 1 epoch: Loss: 1.793255, Acc: 0.560300
Finish 2 epoch: Loss: 0.451671, Acc: 0.865950
Finish 3 epoch: Loss: 0.319043, Acc: 0.902367
Finish 4 epoch: Loss: 0.261764, Acc: 0.920383
Finish 5 epoch: Loss: 0.218886, Acc: 0.933250
Finish 6 epoch: Loss: 0.185247, Acc: 0.943550
Finish 7 epoch: Loss: 0.157959, Acc: 0.952000
Finish 8 epoch: Loss: 0.138422, Acc: 0.958317
Finish 9 epoch: Loss: 0.123145, Acc: 0.962833
Finish 10 epoch: Loss: 0.110414, Acc: 0.966800
Finish 11 epoch: Loss: 0.099997, Acc: 0.969733
Finish 12 epoch: Loss: 0.091435, Acc: 0.972367
Finish 13 epoch: Loss: 0.084382, Acc: 0.974717
Finish 14 epoch: Loss: 0.079316, Acc: 0.976150
Finish 15 epoch: Loss: 0.074298, Acc: 0.977333
Finish 16 epoch: Loss: 0.070559, Acc: 0.978917
Finish 17 epoch: Loss: 0.066490, Acc: 0.979650
Finish 18 epoch: Loss: 0.063585, Acc: 0.980467
Finish 19 epoch: Loss: 0.060467, Acc: 0.980983
Finish 20 epoch: Loss: 0.057499, Acc: 0.982283


<ipython-input-15-9ac4900fd808>:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('cnn.pt')


Test Loss: 0.057923, Acc: 0.981400


In [16]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader

# 定义超参数
batch_size  = 128       # 批的大小

# 下载训练集 MNIST 手写数字测试集
test_dataset  = datasets.MNIST( root='./data', train=False, transform=transforms.ToTensor())
test_loader   = DataLoader(test_dataset , batch_size=batch_size, shuffle=False)

# 加载 Train 模型
model = torch.load('cnn.pt')
criterion = nn.CrossEntropyLoss()
model.eval()
eval_acc  = 0
eval_loss = 0


# 测试
for data in test_loader:
    img, label = data
    if torch.cuda.is_available():
        img   = Variable(img  ).cuda()
        label = Variable(label).cuda()
    else:
        img   = Variable(img  )
        label = Variable(label)

    out  = model(img)
    loss = criterion(out, label)
    eval_loss += loss.item() * label.size(0)

    _ , pred = torch.max(out,1)
    num_correct = (pred==label).sum()
    eval_acc += num_correct.item()
    print('Test Loss: {:.6f}   ,   Acc: {:.6f}'.format( eval_loss/(len(test_dataset)), eval_acc/(len(test_dataset)) ))



<ipython-input-16-84eb8ff2ac16>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('cnn.pt')


Test Loss: 0.000166   ,   Acc: 0.012800
Test Loss: 0.000586   ,   Acc: 0.025400
Test Loss: 0.001389   ,   Acc: 0.037900
Test Loss: 0.002061   ,   Acc: 0.050600
Test Loss: 0.003102   ,   Acc: 0.063000
Test Loss: 0.003962   ,   Acc: 0.075600
Test Loss: 0.004445   ,   Acc: 0.088300
Test Loss: 0.005625   ,   Acc: 0.100800
Test Loss: 0.006492   ,   Acc: 0.113300
Test Loss: 0.008578   ,   Acc: 0.125400
Test Loss: 0.009959   ,   Acc: 0.137800
Test Loss: 0.011108   ,   Acc: 0.150100
Test Loss: 0.011916   ,   Acc: 0.162600
Test Loss: 0.012951   ,   Acc: 0.174900
Test Loss: 0.013783   ,   Acc: 0.187300
Test Loss: 0.014953   ,   Acc: 0.199500
Test Loss: 0.016485   ,   Acc: 0.211800
Test Loss: 0.018048   ,   Acc: 0.224000
Test Loss: 0.019255   ,   Acc: 0.236300
Test Loss: 0.020182   ,   Acc: 0.248700
Test Loss: 0.021641   ,   Acc: 0.261200
Test Loss: 0.022153   ,   Acc: 0.273900
Test Loss: 0.023709   ,   Acc: 0.286300
Test Loss: 0.024604   ,   Acc: 0.298900
Test Loss: 0.025380   ,   Acc: 0.311300


In [17]:
from torchvision import datasets

train_dataset = datasets.MNIST( root='./data', train=True, transform=transforms.ToTensor(), download=True )

